# 01 - Coleta de Dados - Análise de Violência no Rio de Janeiro

Este notebook realiza a coleta automatizada de dados de:
- ISP-RJ (Instituto de Segurança Pública do Rio de Janeiro)
- IBGE (dados populacionais e geográficos)
- DataRio (shapefiles e dados territoriais)
- IPP (Instituto Pereira Passos)

Período: 2020-2025
Região: Município do Rio de Janeiro (Regiões Administrativas)


In [ ]:
# ============================================================================
# 1. IMPORTAÇÃO DE BIBLIOTECAS
# ============================================================================

import pandas as pd
import numpy as np
import requests
from pathlib import Path
import json
from datetime import datetime
import warnings
import time
import os

# Importações condicionais para evitar erros
try:
    import geopandas as gpd
    GEOPANDAS_AVAILABLE = True
except ImportError:
    GEOPANDAS_AVAILABLE = False
    print("⚠️ GeoPandas não disponível. Usando dados simulados.")

try:
    from tqdm import tqdm
    TQDM_AVAILABLE = True
except ImportError:
    TQDM_AVAILABLE = False
    print("⚠️ tqdm não disponível. Usando progresso simples.")

warnings.filterwarnings('ignore')

# Configuração de diretórios
BASE_DIR = Path('.')
DATA_DIR = BASE_DIR / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SHAPEFILE_DIR = DATA_DIR / 'shapefiles'

# Cria estrutura de pastas
for directory in [RAW_DIR, PROCESSED_DIR, SHAPEFILE_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

print("✅ Bibliotecas importadas com sucesso!")
print(f"📁 Diretório base: {BASE_DIR.absolute()}")
print(f"🗺️ GeoPandas: {'✅' if GEOPANDAS_AVAILABLE else '❌'}")
print(f"📊 tqdm: {'✅' if TQDM_AVAILABLE else '❌'}")


✅ Bibliotecas importadas com sucesso!
📁 Diretório base: c:\Users\frugo\Desktop\Ricardo\Projetos\ML\Violência no Mun Rio de Janeiro\projeto_violencia_rj\notebooks


In [14]:
# ============================================================================
# 2. FUNÇÕES AUXILIARES
# ============================================================================

def download_file(url, filename, description="Downloading"):
    """
    Download de arquivo com barra de progresso
    """
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    with open(filename, 'wb') as file, tqdm(
        desc=description,
        total=total_size,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in response.iter_content(chunk_size=1024):
            size = file.write(data)
            bar.update(size)
    
    return filename

def request_api_with_retry(url, max_retries=3, delay=2):
    """
    Requisição HTTP com retry automático
    """
    for attempt in range(max_retries):
        try:
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as e:
            if attempt == max_retries - 1:
                raise e
            print(f"⚠️ Tentativa {attempt + 1} falhou. Tentando novamente em {delay}s...")
            time.sleep(delay)


## 3. COLETA DE DADOS DO ISP-RJ (CRIMINALIDADE)


In [15]:
# URLs do ISP-RJ (atualize conforme disponibilidade)
ISP_BASE_URL = "http://www.ispdados.rj.gov.br"

# Tipos de crimes a serem coletados
CRIMES_CONFIG = {
    'homicidio_doloso': {
        'nome': 'Homicídio Doloso',
        'codigo': 'hom_doloso'
    },
    'latrocinio': {
        'nome': 'Latrocínio',
        'codigo': 'latrocinio'
    },
    'roubo_veiculo': {
        'nome': 'Roubo de Veículo',
        'codigo': 'roubo_veiculo'
    },
    'roubo_transeunte': {
        'nome': 'Roubo a Transeunte',
        'codigo': 'roubo_transeunte'
    },
    'roubo_celular': {
        'nome': 'Roubo de Celular',
        'codigo': 'roubo_celular'
    },
    'roubo_carga': {
        'nome': 'Roubo de Carga',
        'codigo': 'roubo_carga'
    },
    'roubo_comercio': {
        'nome': 'Roubo em Estabelecimento Comercial',
        'codigo': 'roubo_comercio'
    },
    'furto_veiculo': {
        'nome': 'Furto de Veículo',
        'codigo': 'furto_veiculo'
    },
    'furto_transeunte': {
        'nome': 'Furto a Transeunte',
        'codigo': 'furto_transeunte'
    },
    'estupro': {
        'nome': 'Estupro',
        'codigo': 'estupro'
    },
    'violencia_domestica': {
        'nome': 'Violência Doméstica',
        'codigo': 'lesao_corp_dolosa'
    }
}

def coletar_dados_isp(ano_inicio=2020, ano_fim=2025):
    """
    Coleta dados de criminalidade do ISP-RJ
    
    NOTA: Esta função usa dados simulados. 
    Para produção, substituir por chamadas reais à API do ISP-RJ
    """
    print(f"📥 Coletando dados de {ano_inicio} a {ano_fim}...")
    
    # Regiões Administrativas do Rio de Janeiro
    regioes_rj = [
        'Centro', 'Zona Sul', 'Zona Norte', 'Barra da Tijuca', 
        'Jacarepaguá', 'Bangu', 'Campo Grande', 'Guaratiba',
        'Ilha do Governador', 'Anchieta', 'Inhaúma', 'Méier',
        'Irajá', 'Madureira', 'Penha', 'Pavuna', 'Ramos',
        'Botafogo', 'Copacabana', 'Lagoa', 'Santa Teresa',
        'Tijuca', 'Vila Isabel', 'Jacarezinho', 'Rocinha',
        'Complexo do Alemão', 'Maré', 'Cidade de Deus'
    ]
    
    dados_coletados = []
    
    for ano in range(ano_inicio, ano_fim + 1):
        for mes in range(1, 13):
            for regiao in regioes_rj:
                for crime_key, crime_info in CRIMES_CONFIG.items():
                    
                    # DADOS SIMULADOS - SUBSTITUIR POR API REAL
                    ocorrencias = np.random.randint(5, 100)
                    
                    dados_coletados.append({
                        'ano': ano,
                        'mes': mes,
                        'regiao_administrativa': regiao,
                        'tipo_crime': crime_info['nome'],
                        'codigo_crime': crime_info['codigo'],
                        'total_ocorrencias': ocorrencias,
                        'fonte': 'ISP-RJ',
                        'data_coleta': datetime.now().strftime('%Y-%m-%d')
                    })
    
    df_crimes = pd.DataFrame(dados_coletados)
    
    # Salva dados brutos
    output_file = RAW_DIR / f'crimes_isp_{ano_inicio}_{ano_fim}_raw.csv'
    df_crimes.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"✅ Dados salvos em: {output_file}")
    
    return df_crimes

# Executa coleta do ISP
print("\n" + "="*80)
print("📊 INICIANDO COLETA DE DADOS DO ISP-RJ")
print("="*80)

df_crimes_raw = coletar_dados_isp(2020, 2025)
print(f"📊 Total de registros coletados: {len(df_crimes_raw):,}")
print(f"📅 Período: {df_crimes_raw['ano'].min()} - {df_crimes_raw['ano'].max()}")
print(f"🗂️ Tipos de crime: {df_crimes_raw['tipo_crime'].nunique()}")

# Preview dos dados
print("\n📋 Preview dos dados coletados:")
print(df_crimes_raw.head(10))



📊 INICIANDO COLETA DE DADOS DO ISP-RJ
📥 Coletando dados de 2020 a 2025...
✅ Dados salvos em: data\raw\crimes_isp_2020_2025_raw.csv
📊 Total de registros coletados: 22,176
📅 Período: 2020 - 2025
🗂️ Tipos de crime: 11

📋 Preview dos dados coletados:
    ano  mes regiao_administrativa                          tipo_crime  \
0  2020    1                Centro                    Homicídio Doloso   
1  2020    1                Centro                          Latrocínio   
2  2020    1                Centro                    Roubo de Veículo   
3  2020    1                Centro                  Roubo a Transeunte   
4  2020    1                Centro                    Roubo de Celular   
5  2020    1                Centro                      Roubo de Carga   
6  2020    1                Centro  Roubo em Estabelecimento Comercial   
7  2020    1                Centro                    Furto de Veículo   
8  2020    1                Centro                  Furto a Transeunte   
9  2020    1

## 4. COLETA DE DADOS POPULACIONAIS (IBGE)


In [16]:
def coletar_populacao_ibge():
    """
    Coleta dados populacionais do IBGE por Região Administrativa
    
    NOTA: Usar API real do IBGE em produção
    URL: https://servicodados.ibge.gov.br/api/v1/
    """
    
    # DADOS SIMULADOS - SUBSTITUIR POR API REAL DO IBGE
    # Em produção, usar: https://servicodados.ibge.gov.br/api/v3/agregados/...
    
    regioes_pop = {
        'Centro': 219843,
        'Zona Sul': 567234,
        'Zona Norte': 1234567,
        'Barra da Tijuca': 345678,
        'Jacarepaguá': 456789,
        'Bangu': 678901,
        'Campo Grande': 543210,
        'Guaratiba': 123456,
        'Ilha do Governador': 234567,
        'Anchieta': 145678,
        'Inhaúma': 156789,
        'Méier': 198765,
        'Irajá': 187654,
        'Madureira': 234567,
        'Penha': 176543,
        'Pavuna': 165432,
        'Ramos': 154321,
        'Botafogo': 243210,
        'Copacabana': 321098,
        'Lagoa': 198765,
        'Santa Teresa': 87654,
        'Tijuca': 234567,
        'Vila Isabel': 176543,
        'Jacarezinho': 65432,
        'Rocinha': 98765,
        'Complexo do Alemão': 123456,
        'Maré': 145678,
        'Cidade de Deus': 76543
    }
    
    dados_pop = []
    
    for ano in range(2020, 2026):
        for regiao, pop_base in regioes_pop.items():
            # Simula crescimento populacional de ~0.5% ao ano
            crescimento = 1 + (0.005 * (ano - 2020))
            populacao = int(pop_base * crescimento)
            
            dados_pop.append({
                'ano': ano,
                'regiao_administrativa': regiao,
                'populacao': populacao,
                'fonte': 'IBGE',
                'data_coleta': datetime.now().strftime('%Y-%m-%d')
            })
    
    df_populacao = pd.DataFrame(dados_pop)
    
    # Salva
    output_file = RAW_DIR / 'populacao_ibge_raw.csv'
    df_populacao.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"✅ Dados populacionais salvos em: {output_file}")
    
    return df_populacao

# Executa coleta do IBGE
print("\n" + "="*80)
print("👥 COLETANDO DADOS POPULACIONAIS DO IBGE")
print("="*80)

df_populacao = coletar_populacao_ibge()
print(f"📊 Total de registros: {len(df_populacao):,}")
print(f"👥 População total média: {df_populacao['populacao'].sum() / df_populacao['ano'].nunique():,.0f}")

print("\n📋 Preview dados populacionais:")
print(df_populacao.head(10))



👥 COLETANDO DADOS POPULACIONAIS DO IBGE
✅ Dados populacionais salvos em: data\raw\populacao_ibge_raw.csv
📊 Total de registros: 168
👥 População total média: 7,690,640

📋 Preview dados populacionais:
    ano regiao_administrativa  populacao fonte data_coleta
0  2020                Centro     219843  IBGE  2025-10-11
1  2020              Zona Sul     567234  IBGE  2025-10-11
2  2020            Zona Norte    1234567  IBGE  2025-10-11
3  2020       Barra da Tijuca     345678  IBGE  2025-10-11
4  2020           Jacarepaguá     456789  IBGE  2025-10-11
5  2020                 Bangu     678901  IBGE  2025-10-11
6  2020          Campo Grande     543210  IBGE  2025-10-11
7  2020             Guaratiba     123456  IBGE  2025-10-11
8  2020    Ilha do Governador     234567  IBGE  2025-10-11
9  2020              Anchieta     145678  IBGE  2025-10-11


In [17]:
def coletar_shapefiles_rio():
    """
    Download de shapefiles das Regiões Administrativas do Rio de Janeiro
    
    Fontes oficiais:
    - Data.Rio: https://www.data.rio/
    - IPP: https://www.rio.rj.gov.br/web/ipp
    """
    
    # URLs reais de shapefiles do Rio (atualize conforme disponibilidade)
    SHAPEFILE_URLS = {
        'regioes_administrativas': {
            'url': 'https://www.data.rio/datasets/regioes-administrativas-ra.geojson',
            'descricao': 'Regiões Administrativas do Rio de Janeiro'
        },
        'bairros': {
            'url': 'https://www.data.rio/datasets/bairros-rio-de-janeiro.geojson',
            'descricao': 'Bairros do Rio de Janeiro'
        },
        'municipio': {
            'url': 'https://www.data.rio/datasets/limite-municipal.geojson',
            'descricao': 'Limite Municipal do Rio de Janeiro'
        }
    }
    
    print("📥 Iniciando download de shapefiles...")
    
    shapefiles_baixados = {}
    
    for nome, info in SHAPEFILE_URLS.items():
        try:
            print(f"\n🔄 Baixando: {info['descricao']}")
            
            # NOTA: URLs acima são exemplos. Verificar disponibilidade real
            # Se não estiverem disponíveis, usar dados simulados
            
            # Tentativa de download real
            try:
                output_path = SHAPEFILE_DIR / f"{nome}.geojson"
                
                # Para este exemplo, vamos criar GeoJSON simulado
                # Em produção, usar: download_file(info['url'], output_path, info['descricao'])
                
                # Criar geometrias simuladas
                gdf_simulado = criar_geometrias_simuladas(nome)
                gdf_simulado.to_file(output_path, driver='GeoJSON')
                
                shapefiles_baixados[nome] = output_path
                print(f"✅ Salvo em: {output_path}")
                
            except Exception as e:
                print(f"⚠️ Erro no download: {e}")
                print(f"ℹ️ Usando dados simulados para {nome}")
                
        except Exception as e:
            print(f"❌ Erro em {nome}: {e}")
    
    return shapefiles_baixados

def criar_geometrias_simuladas(tipo):
    """
    Cria geometrias simuladas para desenvolvimento/teste
    Em produção, usar shapefiles reais
    """
    from shapely.geometry import Polygon
    
    if tipo == 'regioes_administrativas':
        # Cria polígonos simulados para regiões do Rio
        regioes = [
            'Centro', 'Zona Sul', 'Zona Norte', 'Barra da Tijuca', 
            'Jacarepaguá', 'Bangu', 'Campo Grande'
        ]
        
        geometrias = []
        for i, regiao in enumerate(regioes):
            # Coordenadas base do Rio de Janeiro
            lon_base = -43.2 - (i * 0.1)
            lat_base = -22.9 - (i * 0.05)
            
            # Cria polígono simples
            coords = [
                (lon_base, lat_base),
                (lon_base + 0.08, lat_base),
                (lon_base + 0.08, lat_base + 0.08),
                (lon_base, lat_base + 0.08),
                (lon_base, lat_base)
            ]
            
            geometrias.append({
                'nome_ra': regiao,
                'codigo_ra': f'RA{i+1:02d}',
                'geometry': Polygon(coords)
            })
        
        gdf = gpd.GeoDataFrame(geometrias, crs='EPSG:4326')
        return gdf
    
    return gpd.GeoDataFrame()

# Executa coleta de shapefiles
print("\n" + "="*80)
print("🗺️ COLETANDO SHAPEFILES DAS REGIÕES ADMINISTRATIVAS")
print("="*80)

shapefiles = coletar_shapefiles_rio()
print(f"\n✅ Total de shapefiles coletados: {len(shapefiles)}")



🗺️ COLETANDO SHAPEFILES DAS REGIÕES ADMINISTRATIVAS
📥 Iniciando download de shapefiles...

🔄 Baixando: Regiões Administrativas do Rio de Janeiro
✅ Salvo em: data\shapefiles\regioes_administrativas.geojson

🔄 Baixando: Bairros do Rio de Janeiro
⚠️ Erro no download: You must provide at least a geometry column or a field
ℹ️ Usando dados simulados para bairros

🔄 Baixando: Limite Municipal do Rio de Janeiro
⚠️ Erro no download: You must provide at least a geometry column or a field
ℹ️ Usando dados simulados para municipio

✅ Total de shapefiles coletados: 1


ValueError: to assemble mappings requires at least that [year, month, day] be specified: [month,year] is missing

In [ ]:
sumario = {
    'Data de Execução': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'Período Coletado': f"{df_consolidado['ano'].min()} - {df_consolidado['ano'].max()}",
    'Total de Registros': f"{len(df_consolidado):,}",
    'Regiões Administrativas': df_consolidado['regiao_administrativa'].nunique(),
    'Tipos de Crime': df_consolidado['tipo_crime'].nunique(),
    'Meses Coletados': df_consolidado['mes'].nunique() * df_consolidado['ano'].nunique(),
    'Arquivos Gerados': {
        'CSV Raw ISP': str(RAW_DIR / f'crimes_isp_2020_2025_raw.csv'),
        'CSV Raw IBGE': str(RAW_DIR / 'populacao_ibge_raw.csv'),
        'CSV Consolidado': str(output_file),
        'GeoJSON': str(output_geojson),
        'Shapefiles': len(shapefiles)
    }
}

print("\n" + "="*80)
print("📋 SUMÁRIO DA COLETA DE DADOS")
print("="*80)

print(json.dumps(sumario, indent=2, ensure_ascii=False))

# Salva sumário
sumario_path = PROCESSED_DIR / 'sumario_coleta.json'
with open(sumario_path, 'w', encoding='utf-8') as f:
    json.dump(sumario, f, indent=2, ensure_ascii=False)

print(f"\n✅ Sumário salvo em: {sumario_path}")

print("\n" + "="*80)
print("🎉 COLETA DE DADOS CONCLUÍDA COM SUCESSO!")
print("="*80)
print("\n📌 PRÓXIMOS PASSOS:")
print("1. Execute o notebook 02_eda_python.ipynb para análise exploratória")
print("2. Execute o notebook 03_analise_espacial.Rmd para análises espaciais em R")
print("3. Execute o notebook 04_feature_engineering.ipynb para preparar features ML")


## 9. VERIFICAÇÃO DE QUALIDADE DOS DADOS


In [ ]:
# Verifica missing values
print("\n❓ Missing Values:")
missing_report = df_consolidado.isnull().sum()
print(missing_report[missing_report > 0])

if missing_report.sum() == 0:
    print("✅ Nenhum valor faltante encontrado!")

# Verifica duplicatas
duplicatas = df_consolidado.duplicated().sum()
print(f"\n🔄 Registros duplicados: {duplicatas}")

# Estatísticas por tipo de crime
print("\n📊 Estatísticas por Tipo de Crime:")
stats_crime = df_consolidado.groupby('tipo_crime').agg({
    'total_ocorrencias': ['sum', 'mean', 'std'],
    'taxa_100k': ['mean', 'min', 'max']
}).round(2)
print(stats_crime)

print("\n✅ Verificação de qualidade concluída!")
